In [3]:
import numpy as np
import cv2
import pyautogui
ob=cv2.VideoCapture(0)
start=0
flag=0
while True:    
    if not ob.isOpened():
        ob.open()   
    _, im=ob.read()
       
    lower=np.array([0,48,80], dtype="uint8")
    upper=np.array([20,255,255], dtype="uint8")
    
    img=im
    converted=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    skinmask=cv2.inRange(converted, lower, upper)
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    skinmask=cv2.erode(skinmask, kernel, iterations=1)
    skinmask=cv2.dilate(skinmask, kernel, iterations=2)
    skinmask=cv2.GaussianBlur(skinmask, (3,3), 0)
    im=cv2.bitwise_and(img, img, mask=skinmask)
    
    imgray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret,thresh=cv2.threshold(imgray,50,255,0)
    contours, hierarchy=cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    max_i=0
    max_area=0
    for i in range(len(contours)):
        c=contours[i]
        area_cnt=cv2.contourArea(c)
        if area_cnt>max_area:
            max_area=area_cnt
            max_i=i
            
    if max_i<len(contours):
        max_c=contours[max_i]
        moment=cv2.moments(max_c)
        if moment['m00'] != 0:
            gx=int(moment['m10']/moment['m00'])
            gy=int(moment['m01']/moment['m00'])
        g=gx,gy

        cv2.drawContours(im, [max_c], 0, (0,255,0), 3)
        cv2.circle(im, g, 5,(0,0,255), -1)
        cv2.line(im, (0, int(im.shape[0]/2)+40), (im.shape[1], int(im.shape[0]/2)+40), (255,0,0))
        cv2.line(im, (0, int(im.shape[0]/2)-40), (im.shape[1], int(im.shape[0]/2)-40), (255,0,0))
        cv2.imshow("Dino", im)
    
    if gy<int(im.shape[0]/2)+40 and gy>int(im.shape[0]/2)-40 :
        if start==0 :
            pyautogui.press("space")
            start=1
        else:
            flag=0
    elif flag==0:
        pyautogui.press("space")
        flag=1
    
    if cv2.waitKey(1) &0xFF==ord('x'):
        break

cv2.waitKey(0)
ob.release()
cv2.destroyAllWindows()